In [1]:
import random
from copy import deepcopy
from networkx import Graph, minimum_spanning_tree
import networkx as nx

class Individual:
    def __init__(self, graph, edges = None):
        self.graph = graph
        self.edges = edges or self.initialize_spanning_tree()
        self.fitness = self.calc_fitness()
    
    def initialize_spanning_tree(self):
        mst = minimum_spanning_tree(self.graph)
        return mst.edges
    
    def calc_fitness(self):
        self.graph.add_edges_from(self.edges)
        return sum(1 for node in self.graph.nodes if self.graph.degree[node] == 1)

In [2]:

def crossover(parent_1, parent_2):    
    combined_edges = list(set(parent_1.edges) | set(parent_2.edges))
    
    temp_graph = Graph()
    temp_graph.add_edges_from(combined_edges)
    child_edges = list(minimum_spanning_tree(temp_graph).edges)

    return Individual(Graph(), child_edges)

In [3]:
def mutation(individual, prob):
    if random.random() < prob:
        all_edges = list(individual.graph.edges)
        individual.edges[random.randint(0, len(individual.edges) - 1)] = random.choice(all_edges)
        individual.fitness = individual.calc_fitness()

In [4]:
def selection(population, tournament_size):
      participants = random.sample(population, tournament_size)
      return max(participants, key=lambda x: x.fitness)

In [5]:
def ga(graph, num_iters, elitism_size, prob, population_size, tournament_size):

    population = [Individual(graph) for _ in range(population_size)]
    new_population = deepcopy(population)
    
    if elitism_size % 2 != len(population) % 2:
        elitism_size += 1
    
    for _ in range(num_iters):
        population.sort(key=lambda x: x.fitness, reverse=True)
        new_population[:elitism_size] = deepcopy(population[:elitism_size])
    
    for i in range(elitism_size, population_size, 2):
        parent_1 = selection(population, tournament_size)
        parent_2 = selection(population, tournament_size)
        
        child_1 = crossover(parent_1, parent_2)
        child_2 = crossover(parent_1, parent_2)
        
        mutation(child_1, prob)
        mutation(child_2, prob)
        
        new_population[i] = child_1
        new_population[i + 1] = child_2
        
        population = deepcopy(new_population)
    
    return max(population, key=lambda x: x.fitness)